<a href="https://colab.research.google.com/github/habiclaude88/Sentiment_Analysis/blob/main/NLPNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.chdir("/content/gdrive/MyDrive/NLP FELLOWSHIP/WEEK5")
!ls

50k_imdb_movie_reviews.csv  NLPNN.ipynb


In [19]:
import pandas as pd

full_dataset = pd.read_csv('50k_imdb_movie_reviews.csv')
full_dataset.sample(20)

,review,sentiment,set
43972,At first glance this documentary/fiction/carto...,0,train
20718,Scarier than any horror movie ever made becaus...,0,test
7504,I think this movie is the most misunderstood f...,1,test
49688,This movie is just boring.<br /><br />It tries...,0,train
49545,"I've seen the movie only recently, although it...",0,train
25317,This movie is amazing! While being funny and e...,1,train
1602,This film gave me probably the most pleasant s...,1,test
9393,I was lucky to see this sequel before the orig...,1,test
49192,This sequel is a total rehash of the first fil...,0,train
21397,"First of all, Katherine Hepburn is badly misca...",0,test


In [6]:
print("No of positive in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'train')])))
print("No of negative in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'train')])))
print("No of positive in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'test')])))
print("No of negative in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'test')])))

No of positive in train : 12500
No of negative in train : 12500
No of positive in test : 12500
No of negative in test : 12500


In [7]:
full_dataset['review'].describe()

count                                                 50000
unique                                                49582
top       Loved today's show!!! It was a variety and not...
freq                                                      5
Name: review, dtype: object

In [8]:
full_dataset = full_dataset.drop_duplicates(subset=['review'])

In [9]:
full_dataset['review'].describe()

count                                                 49582
unique                                                49582
top       I went and saw this movie last night after bei...
freq                                                      1
Name: review, dtype: object

In [10]:
print("No of positive in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'train')])))
print("No of negative in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'train')])))
print("No of positive in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'test')])))
print("No of negative in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'test')])))

No of positive in train : 12444
No of negative in train : 12337
No of positive in test : 12440
No of negative in test : 12361


In [11]:
train_dataset = full_dataset[(full_dataset['set'] == 'train')][['review','sentiment']]
test_dataset = full_dataset[(full_dataset['set'] == 'test')][['review','sentiment']]
test_dataset.head()

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


In [12]:
import re
def preprocessing(texts):
  cleaned_text = []
  for text in texts:
    text = text.lower()
    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                "]+", flags=re.UNICODE)
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    html_pattern = re.compile('<.*?>')
    text = emoji_pattern.sub(r'', text)
    text = url_pattern.sub(r'', text)
    text = html_pattern.sub(r'', text)
    text = re.sub(r"[^\w\d'\s]+", ' ', text)
    cleaned_text.append(text)

  return cleaned_text

In [13]:
!pip install torch==1.8.0 torchtext==0.9.0 #compatibility

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 735.5 MB 10 kB/s 
     |████████████████████████████████| 7.1 MB 20.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.8.0 which is incompatible.


In [14]:
import torch
from torchtext.legacy import data
from torchtext.legacy.data import Dataset, Example
from torchtext.legacy.data import BucketIterator


SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True # Check this
max_document_length = 300 #hyperparameter

TEXT = data.Field(lower=True, include_lengths=True,  tokenize='spacy',preprocessing=preprocessing,batch_first=True,  fix_length=max_document_length)
LABEL = data.Field(sequential=False, use_vocab=False)

class DataFrameDataset(Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )



/usr/local/lib/python3.7/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [20]:
torch_valid_dataset, torch_test_dataset = DataFrameDataset(
    df=test_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
).split() 

In [21]:
torch_train_dataset = DataFrameDataset(
    df=train_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
)

In [22]:
max_size = 30000 #hyperparameter
TEXT.build_vocab(torch_train_dataset, max_size=max_size,vectors='fasttext.simple.300d')
vocab_size = len(TEXT.vocab)


.vector_cache/wiki.simple.vec: 293MB [00:15, 18.4MB/s]                           
100%|██████████| 111051/111051 [00:14<00:00, 7672.30it/s]


In [23]:
print(TEXT.vocab.freqs.most_common(20))

[(' ', 785692), ('the', 326108), ('and', 161663), ('a', 160563), ('of', 144477), ('to', 134325), ('is', 109343), ('it', 92798), ('in', 92099), ('i', 81980), ('this', 72819), ('that', 72538), ("'s", 61640), (' br', 52238), ('was', 49935), ('as', 45797), ('for', 43582), ('with', 43503), ('movie', 42699), ('but', 41298)]


In [24]:
BATCH_SIZE = 64 #hyperparameter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (torch_train_dataset, torch_valid_dataset, torch_test_dataset), 
    batch_size = BATCH_SIZE ,
    sort_key=lambda x: len(x.review),
    sort_within_batch=True,
    device = device)

In [25]:
import torch.nn as nn
import torch.nn.functional as F


class LR(nn.Module):
    def __init__(self, input_size, hidden_size,hidden_size2, num_classes):
        super(LR, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size2) 
        self.fc3 = nn.Linear(hidden_size2, num_classes)

    def forward(self, text):
        text = text.float() # dense layer deals just with float type data
        x = self.fc1(text) #(m x n) with (n x p)
        x = self.relu(x)
        x = self.relu(self.fc2(x))
        
        preds = self.fc3(x) # crossentropyloss handles the softmax
        # preds = F.softmax(preds,1) # nn.softmax
        return preds

In [29]:
lr = 1e-4
batch_size = 64
dropout_keep_prob = 0.5
embedding_size = 300
max_document_length = 300 # each sentence has until 100 words
vocab_size = len(TEXT.vocab)
dev_size = 0.8 # split percentage to train\validation data
max_size = 30000 # maximum vocabulary size
seed = 42
num_classes = 2

num_epochs = 10
hidden_size = 256
hidden_size1 = 300
hidden_size2 = 128
hidden_size3 = 64

to_train = True

model = LR(max_document_length, hidden_size,hidden_size2, num_classes)


In [27]:
def accuracy(probs, target):
  winners = probs.argmax(dim=1)
  corrects = (winners == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy

In [28]:
best_valid_loss = float('inf')
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

loss_func = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
  train_epoch_loss = 0
  train_epoch_acc = 0
  for batch in train_iterator:
      optimizer.zero_grad()
      # retrieve text and no. of words
      text, text_lengths = batch.review

      #feedforward
      predictions = model(text).squeeze(1)
      
      
      loss = loss_func(predictions, batch.sentiment)

      acc = accuracy(predictions, batch.sentiment)

      # perform backpropagation
      loss.backward()

      optimizer.step()

      train_epoch_loss += loss.item()
      train_epoch_acc += acc.item()

  

  valid_epoch_loss = 0
  valid_epoch_acc = 0

  model.eval()

  with torch.no_grad():
      for batch in valid_iterator:
          text, text_lengths = batch.review

          predictions = model(text).squeeze(1)

          loss = loss_func(predictions, batch.sentiment)

          acc = accuracy(predictions, batch.sentiment)

          valid_epoch_loss += loss.item()
          valid_epoch_acc += acc.item()

   

  if valid_epoch_loss < best_valid_loss:
            best_valid_loss = valid_epoch_loss
            torch.save(model.state_dict(), 'saved_weights'+'_linear.pt')

  print(f'\tTrain Loss: {train_epoch_loss / len(train_iterator):.3f} | Train Acc: {train_epoch_acc  / len(train_iterator)* 100:.2f}%')
  print(f'\t Val. Loss: {valid_epoch_loss / len(valid_iterator):.3f} |  Val. Acc: {valid_epoch_acc / len(valid_iterator) * 100:.2f}%')


	Train Loss: 62.447 | Train Acc: 49.99%
	 Val. Loss: 37.171 |  Val. Acc: 50.56%
	Train Loss: 26.326 | Train Acc: 55.04%
	 Val. Loss: 26.508 |  Val. Acc: 50.31%
	Train Loss: 15.152 | Train Acc: 59.28%
	 Val. Loss: 21.849 |  Val. Acc: 49.95%
	Train Loss: 9.349 | Train Acc: 64.73%
	 Val. Loss: 18.974 |  Val. Acc: 50.34%
	Train Loss: 5.940 | Train Acc: 69.27%
	 Val. Loss: 16.534 |  Val. Acc: 50.60%
	Train Loss: 3.947 | Train Acc: 73.25%
	 Val. Loss: 15.231 |  Val. Acc: 50.23%
	Train Loss: 2.615 | Train Acc: 77.55%
	 Val. Loss: 14.292 |  Val. Acc: 50.16%
	Train Loss: 1.822 | Train Acc: 80.64%
	 Val. Loss: 13.295 |  Val. Acc: 50.16%
	Train Loss: 1.255 | Train Acc: 83.47%
	 Val. Loss: 12.697 |  Val. Acc: 50.34%
	Train Loss: 0.934 | Train Acc: 85.72%
	 Val. Loss: 12.378 |  Val. Acc: 50.26%
